In [2]:
import os
import random
import logging
from copy import deepcopy
from collections import defaultdict
from matplotlib import pyplot as plt

import cv2
import torch
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader

from isegm.utils.log import logger, TqdmToLogger, SummaryWriterAvg
from isegm.utils.vis import draw_probmap, draw_points, add_tag
from isegm.utils.misc import save_checkpoint
from isegm.utils.serialization import get_config_repr
from isegm.utils.distributed import get_dp_wrapper, get_sampler, reduce_loss_dict
from torch.cuda.amp import autocast as autocast, GradScaler

from isegm.utils.exp_imports.default import * 

MODEL_NAME = "lidc_hrnet32"

from isegm.data.compose import ComposeDataset, ProportionalComposeDataset
import torch.nn as nn
from isegm.data.aligned_augmentation import AlignedAugmentator
# from isegm.engine.focalclick_trainer import ISTrainer

from isegm.data.compose import ComposeDataset, ProportionalComposeDataset
import torch.nn as nn
from isegm.data.aligned_augmentation import AlignedAugmentator
# from isegm.engine.focalclick_trainer import ISTrainer
from isegm.data.preprocess import Preprocessor
from isegm.utils import exp
from isegm.inference import utils

cfg = exp.load_config_file('config.yml', return_edict=True)

In [3]:

points_sampler = MultiPointSampler(
        24,
        prob_gamma=0.80,
        merge_objects_prob=0.15,
        max_num_merged_objects=2,
        use_hierarchy=False,
        first_click_center=True,
    )

preprocessor = Preprocessor()

dataset = utils.get_dataset('LIDC_2D_VAL', cfg, preprocessor=preprocessor)
# dataset = utils.get_dataset('LIDC_CROPS_VAL', cfg, preprocessor=preprocessor)
dataset.points_sampler = points_sampler

In [4]:
len(dataset)

57

In [5]:
import cv2
def get_box_corners(mask):
    mask = mask.astype(np.uint8)
    mask[mask > 0] = 1
    mask = mask.astype(np.uint8)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    x, y, w, h = cv2.boundingRect(contours[0])
    return x, y, x + w, y + h

In [6]:
IMAGE_HEIGHT, IMAGE_WIDTH = dataset.get_sample(0).image.shape[:2]
PIXEL_COUNT = IMAGE_HEIGHT * IMAGE_WIDTH
PIXEL_COUNT

262144

In [7]:
box_sizes = []
widths = []
heights = []

for i in range(len(dataset)):
# for i in range(int(len(dataset) * 0.15)):
    sample = dataset.get_sample(i)
    mask = sample.gt_mask
    x1, y1, x2, y2 = get_box_corners(mask)
    box_size = (x2 - x1) * (y2 - y1) * 1.5 # 1.5 as click is not expected to be on the mask 
    box_sizes.append(box_size)
    widths.append(x2 - x1)
    heights.append(y2 - y1)
    
mean_box_size = np.mean(box_sizes)
mean_width = np.mean(widths)
mean_height = np.mean(heights)
mean_box_size, mean_width, mean_height

(538.078947368421, 16.789473684210527, 16.63157894736842)

In [8]:
box_percentage = mean_box_size / PIXEL_COUNT
width_percentage = mean_width / IMAGE_WIDTH
height_percentage = mean_height / IMAGE_HEIGHT
box_percentage, width_percentage, height_percentage

(0.002052608289216694, 0.032791940789473686, 0.03248355263157895)

In [9]:
average_ratio = (width_percentage + height_percentage) / 2
average_ratio

0.032637746710526314

In [10]:
bounds = np.round(average_ratio * 0.5, 3), np.round(average_ratio * 1.5, 3)
print(bounds)
print(f'*6:{bounds[0] * 4} : {bounds[1] * 4}')
print(f'x10.5:{bounds[0] * 7} : {bounds[1] * 7}')
#LIDC 2D VAL: (0.015, 0.044)

(0.016, 0.049)
*6:0.064 : 0.196
x10.5:0.112 : 0.343


In [12]:
bounds[0]*4, bounds[1]*4

(0.064, 0.196)